# Instancing

In [ ]:
# Assignment 7 Remi Roper

In [ ]:
import glm
#from objloader import Obj
import numpy as np
import pygame
import moderngl
from objloader import Obj
from math import cos, sin, sqrt
import ctypes
#ctypes.windll.user32.SetProcessDPIAware()

### Read data from an OBJ modelfile

In [ ]:
class SceneBound:
    def __init__(self, coords):
        self.boundingBox = [
        np.min(coords,0),
        np.max(coords,0)
        ]
        self.center = (self.boundingBox[0] + self.boundingBox[1])/2
        dVector = (self.boundingBox[1] - self.boundingBox[0])
        self.radius = sqrt(dVector[0]*dVector[0] + dVector[1]*dVector[1]+dVector[2]*dVector[2])/2
    def __str__(self):
        return f"boundingBox:{self.boundingBox}, enter: {self.center}, Radius:{self.radius}."

def get_triangle_normal(vertexList):
    e1 = glm.vec3(vertexList[1]) - glm.vec3(vertexList[0])
    e2 = glm.vec3(vertexList[2]) - glm.vec3(vertexList[0])
    return list(glm.normalize(glm.cross(e1, e2)))

def compute_triangle_normal_coords(position_coord):
    i = 0
    normals = []
    while i < len(position_coord):
        normal = get_triangle_normal(position_coord[i:i+3])
        normals.append(normal)
        normals.append(normal)
        normals.append(normal)
        i += 3
    return np.array(normals)

def getObjectData(filePath, normal=False, texture = False):
    geometry = Obj.open(filePath)
    position_coord = np.array([geometry.vert[f[0]-1] for f in geometry.face])
    if normal==True:
        if geometry.norm:
            normal_coord = np.array([geometry.norm[f[2]-1] for f in geometry.face])
            print("Normal exists")
        else:
            normal_coord = compute_triangle_normal_coords(position_coord)
            print("Normal computed.")
    if texture==True:
        if geometry.text:
            texture_coord = np.array([[geometry.text[f[1]-1][0],geometry.text[f[1]-1][1]] for f in geometry.face])
            print ("texture exists")
        else:
            texture_coord = np.array([[0.5,0.5] for f in geometry.face])
            print("No texture")
    if (normal==False and texture == False):
        vertex_data = position_coord.astype("float32").flatten()
    elif texture == False:
        vertex_data = np.concatenate((position_coord,normal_coord),axis=1).astype("float32").flatten()
    elif normal == False:
        vertex_data = np.concatenate((position_coord,texture_coord),axis=1).astype("float32").flatten()
    else:
        vertex_data = np.concatenate((position_coord,normal_coord,texture_coord),axis=1).astype("float32").flatten()
    return [vertex_data, SceneBound(position_coord)]


In [ ]:
files = ["teapot_with_texCoords-1.obj", "cube.obj", "20_icosahedron.obj"] # Teapot, floor, and light source

In [ ]:
teapot_vertex_data, teapotBound = getObjectData(files[0], normal=True)
floor_vertex_data, floorBound = getObjectData(files[1], normal=True)
light_vertex_data, lightBound = getObjectData(files[2], normal=True)

In [ ]:
floor_object_transformation = glm.scale(glm.vec3(10,0.05,10))*glm.translate(glm.vec3(0,-1,0))
teapot_object_transformation = glm.scale(glm.vec3(0.3))*glm.translate(glm.vec3(0,7.875,0))
light_object_transformation = glm.scale(glm.vec3(0.1))

### Initialize pygame and create a window with OpenGL context.

In [ ]:
pygame.init() # Initlizes its different modules. Display module is one of them.
clock = pygame.time.Clock()
window = pygame.display.set_mode((1000, 800), flags= pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE , vsync=True) 
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MAJOR_VERSION,4)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MINOR_VERSION,1)
pygame.display.set_caption(title = "Assignment 7: Remi Roper")
gl = moderngl.get_context()  # Get Previously created context.
gl.enable(gl.DEPTH_TEST)
width, height = window.get_size()
aspect_ratio = width/height

### Push the Geometry Data to the GPU buffer.

In [ ]:
teapot_vertex_buffer = gl.buffer(teapot_vertex_data)
floor_vertex_buffer = gl.buffer(floor_vertex_data)
light_vertex_buffer = gl.buffer(light_vertex_data)

### Write shader code and Create shader program

In [ ]:
vertex_shader_code = '''#version 330 core

layout (location = 0) in vec4 in_position;
layout (location = 1) in vec3 in_normal;

uniform mat4 model;
uniform mat4 view, perspective;

out vec3 f_normal;

void main() {
    vec4 world_position = model*in_position;
    gl_Position = perspective*view*world_position;
    f_normal = in_normal;

    mat3 normalMatrix = mat3(transpose(inverse(model)));
    f_normal = normalize(normalMatrix*in_normal);
}
'''
fragment_shader_code ='''
#version 330 core

in vec3 f_normal;
in vec3 f_position;
out vec4 outColor;
uniform vec3 materialColor;
uniform vec4 light;

vec3 computeColor(){
    vec3 lightVector = light.xyz;
    if (light.w > 0)
        lightVector = light.xyz - f_position;
    vec3 L = normalize(lightVector);
    vec3 N = normalize(f_normal);
    return materialColor * clamp(dot(L,N),0.,1.);
}

void main()
{
    vec3 color = computeColor();
    outColor = vec4 (color, 1);
}

'''

fragment_shader_code_for_light_source ='''#version 330 core

in vec3 f_position;
in vec3 f_normal;

layout (location = 0) out vec4 out_color;

void main() {
    vec3 N = normalize(f_normal);
    out_color = vec4(0.5*(N+1.0),1.0);
}'''

sceneProgram = gl.program(
    vertex_shader=vertex_shader_code,
    fragment_shader=fragment_shader_code
)

lightProgram = gl.program(
    vertex_shader=vertex_shader_code,
    fragment_shader=fragment_shader_code_for_light_source
)

In [ ]:
def viewProgramParameters(program):
    for name in program:
        member = program[name]
        print(name, type(member), member)

In [ ]:
viewProgramParameters(lightProgram)

In [ ]:
viewProgramParameters(sceneProgram)

### Connect the buffers to shader program

In [ ]:
#https://moderngl.readthedocs.io/en/5.10.0/topics/buffer_format.html#syntax
teapot_renderable = gl.vertex_array(sceneProgram, [
    (teapot_vertex_buffer, '3f 3f', 'in_position', 'in_normal')
]) #for vertex_buffer in vertex_buffers]

floor_renderable = gl.vertex_array(sceneProgram, [
    (floor_vertex_buffer, '3f 3f', 'in_position', 'in_normal')
])

light_renderable = gl.vertex_array(lightProgram, [
    (light_vertex_buffer, '3f 3f', 'in_position', 'in_normal')
])

In [ ]:
near = 1
far = 45
aspect_ratio = width/height

perspectiveMatrix = glm.perspective(glm.radians(60.0), aspect_ratio, near, far)

target = glm.vec3(0)
cameraDistance = 20
eye = target + cameraDistance*glm.normalize(glm.vec3(0,10,20))
upVector = glm.vec3(0.0, 1.0, 0.0)
viewMatrix = glm.lookAt(eye, target, upVector)
lviewMatrix = viewMatrix

angle_in_radian = pygame.time.get_ticks() / 1500.0
rotateMatrix = glm.rotate(glm.mat4(1), glm.radians(angle_in_radian), glm.vec3(0, 1 ,0))

In [ ]:
# start light vector 
lightDistance = 12
lightVector = glm.normalize(glm.vec3(-1,1,0))# This will change for dynamic light

In [ ]:
running = True
depth = True
toggle = True
angle = 0

FLOOR_COLOR = glm.vec3(0.0,0.6,0.0)
TP_COLOR = glm.vec3(1.0,0.7,0.5)

sceneProgram['view'].write(viewMatrix)
sceneProgram["perspective"].write(perspectiveMatrix)
lightProgram['view'].write(lviewMatrix)
lightProgram["perspective"].write(perspectiveMatrix)

lightPosition = lightDistance*lightVector
newLight = glm.vec4(lightPosition, 1.0)
newLight = light_object_transformation * newLight
sceneProgram['light'].write(newLight)


while running:   
    clock.tick(60)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif (event.type == pygame.KEYDOWN):
            if event.key == 27:
                running = False
            elif event.key == pygame.K_d:
                depth = not depth
                if depth:
                    gl.enable(gl.DEPTH_TEST)
                else:
                    gl.disable(gl.DEPTH_TEST)
            elif event.key == pygame.K_l:
                toggle = not toggle
        elif (event.type == pygame.WINDOWRESIZED):
            aspect_ratio = event.x/event.y
            perspectiveMatrix = glm.perspective(glm.radians(60.0), aspect_ratio, near, far)
            sceneProgram["perspective"].write(perspectiveMatrix) 
            lightProgram["perspective"].write(perspectiveMatrix)
            
    gl.clear(0.1,0.1,0.1,depth=1.0)

    # For orbiting light you must update the light direction every frame and recompute the light position and light_model_transformation.
    # 
    lightPosition = lightDistance*lightVector # Light is placed lightDistance away from the world Origin along the lightVector direction
    light_model_transformation = glm.translate(lightPosition)*light_object_transformation

    lviewMatrix = lviewMatrix * rotateMatrix
    lightProgram['view'].write(lviewMatrix)

    if toggle:
        newLight = glm.vec4(lightPosition, 1.0)
        newLight = lviewMatrix *  perspectiveMatrix * newLight 
    else:
        newLight = glm.vec4(lightPosition, 1.0)
    
    sceneProgram['light'].write(newLight)
    lightProgram["model"].write(light_model_transformation)
    light_renderable.render()

    sceneProgram["model"].write(floor_object_transformation)
    sceneProgram["materialColor"].write(FLOOR_COLOR)
    floor_renderable.render()

    sceneProgram["model"].write(teapot_object_transformation)
    sceneProgram["materialColor"].write(TP_COLOR)
    teapot_renderable.render()

    pygame.display.flip()

In [ ]:
pygame.quit()